# Convolutional Neural Network

### Importing the libraries

In [1]:
!pip install --upgrade tensorflow keras Pillow

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: tensorflow in /home/matheushenrick/.local/lib/python2.7/site-packages (2.1.0)
Requirement already up-to-date: keras in /home/matheushenrick/.local/lib/python2.7/site-packages (2.8.0)
     |████████████████████████████████| 2.1 MB 2.1 MB/s eta 0:00:01
Processing /home/matheushenrick/.cache/pip/wheels/0f/10/f7/29260ef8a721b90061c8c70a4f0130a64036e8dafe15acc097/gast-0.2.2-py2-none-any.whl
  Using cached scipy-1.2.2-cp27-cp27mu-manylinux1_x86_64.whl (24.8 MB)
  Attempting uninstall: gast
    Found exis

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

2022-03-24 15:45:49.406192: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-24 15:45:49.406214: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
tf.__version__

'2.8.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

2022-03-24 15:46:08.042640: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-24 15:46:08.042663: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-24 15:46:08.042677: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (matheushenrick-PC): /proc/driver/nvidia/version does not exist
2022-03-24 15:46:08.042855: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='leaky_relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='leaky_relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='leaky_relu'))
cnn.add(tf.keras.layers.Dense(units=64, activation='leaky_relu'))
cnn.add(tf.keras.layers.Dense(units=32, activation='leaky_relu'))

### Step 5 - Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 41s 162ms/step - loss: 0.6800 - accuracy: 0.5745 - val_loss: 0.6267 - val_accuracy: 0.6650
Epoch 2/25
250/250 [==============================] - 43s 172ms/step - loss: 0.5968 - accuracy: 0.6780 - val_loss: 0.5481 - val_accuracy: 0.7315
Epoch 3/25
250/250 [==============================] - 43s 172ms/step - loss: 0.5451 - accuracy: 0.7199 - val_loss: 0.5412 - val_accuracy: 0.7230
Epoch 4/25
250/250 [==============================] - 43s 172ms/step - loss: 0.5161 - accuracy: 0.7389 - val_loss: 0.5055 - val_accuracy: 0.7575
Epoch 5/25
250/250 [==============================] - 43s 173ms/step - loss: 0.4938 - accuracy: 0.7540 - val_loss: 0.5034 - val_accuracy: 0.7660
Epoch 6/25
250/250 [==============================] - 43s 174ms/step - loss: 0.4712 - accuracy: 0.7761 - val_loss: 0.5221 - val_accuracy: 0.7515
Epoch 7/25
250/250 [==============================] - 44s 176ms/step - loss: 0.4541 - accuracy: 0.7793 - val_loss: 0.4880 - val_ac

## Part 4 - Making a single prediction

In [16]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [17]:
print(prediction)

dog


## Fim